In [1]:
import pandas as pd
import numpy as np
import datetime
import re
import matplotlib.pyplot as plt
import seaborn as sns

# Data Cleaning

Source data: Computer-Aided Dispatch data from EPD dispatch between January 1, 2014 and April 18, 2025.

### Steps for cleaning

1. Studying the format, entries, and each variable’s data type, inconsistencies, and formatting errors.
2. Recoding the dataset by: (1) stripping all columns with string values, (2) using the datetime package to convert the calltime variable to datetime objects, and (3) creating a new variable, agency, using boolean logic that indicates whether CAHOOTS or EPD responds to a call.
3. Conducting visual and statistical analysis of the dataset to make sure I understand variable distributions, as well as to find outliers and missing values.
4. Fixing identified outliers by: (1) determining if each outlier is a true outlier or an error in date entry, (2) removing/correcting outliers when appropriate, and (3) re-plotting distributions and/or computing updated summary statistics.
5. Doing a final check for consistency in the dataframe by: (1) checking that each variable is of the correct data type, (2) making sure missing values are intentional or properly handled, (3) using .describe to make sure that numerical variables are within the expected ranges, and (4) checking for consistent categorical and date/time values.

### Import CAD data from 2014-2025
Import each years .csv file into a separate, corresponding DataFrame. Concatenate each individual DataFrame into the master DataFrame: `cad`.

In [2]:
cad14 = pd.read_csv("../data/class_data_2014.csv", dtype={4: str, 16: str, 18: str})
cad15 = pd.read_csv("../data/class_data_2015.csv", dtype={4: str, 16: str, 18: str})
cad16 = pd.read_csv("../data/class_data_2016.csv", dtype={4: str, 16: str, 18: str})
cad17 = pd.read_csv("../data/class_data_2017.csv", dtype={4: str, 16: str, 18: str})
cad18 = pd.read_csv("../data/class_data_2018.csv", dtype={4: str, 16: str, 18: str})
cad19 = pd.read_csv("../data/class_data_2019.csv", dtype={4: str, 16: str, 18: str})
cad20 = pd.read_csv("../data/class_data_2020.csv", dtype={4: str, 16: str, 18: str})
cad21 = pd.read_csv("../data/class_data_2021.csv", dtype={4: str, 16: str, 18: str})
cad22 = pd.read_csv("../data/class_data_2022.csv", dtype={4: str, 16: str, 18: str})
cad23 = pd.read_csv("../data/class_data_2023.csv", dtype={4: str, 16: str, 18: str})
cad24 = pd.read_csv("../data/class_data_2024.csv", dtype={4: str, 16: str, 18: str})
cad25 = pd.read_csv("../data/class_data_2025.csv", dtype={4: str, 16: str, 18: str})

cad = pd.concat([cad14, cad15, cad16, cad17, cad18, cad19, cad20, cad21, cad22, cad23, cad24, cad25], ignore_index=True)

#### Begin data cleaning steps by creating `clean`, a copy of the original DataFrame.

In [3]:
clean = cad.copy()

### Recoding

#### Strip str variables of extra spaces/blanks.

In [4]:
clean.loc[:, 'closecode'] = clean['closecode'].str.strip()
clean.loc[:, 'service'] = clean['service'].str.strip()
clean.loc[:, 'callsource'] = clean['callsource'].str.strip()
clean.loc[:, 'nature'] = clean['nature'].str.strip()
clean.loc[:, 'closed_as'] = clean['closed_as'].str.strip()
clean.loc[:, 'beat'] = clean['beat'].str.strip()
clean.loc[:, 'primeunit'] = clean['primeunit'].str.strip()
clean.loc[:, 'priority'] = clean['priority'].str.strip()
clean.loc[:, 'zip'] = clean['zip'].str.strip()

#### Convert numbered variables to `int` values.

In [5]:
clean.loc[:, 'inci_id'] = clean['inci_id'].astype(int)
clean.loc[:, 'units_dispd'] = clean['units_dispd'].astype(int)
clean.loc[:, 'units_arrived'] = clean['units_arrived'].astype(int)

#### Convert time-based variables to datetime objects.

In [6]:
clean.loc[:, 'calltime'] = pd.to_datetime(clean['calltime'], format="%Y-%m-%d %H:%M:%S.%f")

#### Create agency column using `primeunit`. If `primeunit` matches one of the CAHOOTS units, then agency is assigned to `CAHOOTS`, otherwise it is `EPD`.

In [7]:
cahoots_pattern = r"1J77\s*|3J79\s*|3J78\s*|3J77\s*|4J79\s*|3J81\s*|3J76\s*|2J28\s*|2J29\s*|CAHOOT\s*|CAHOT\s*|CAHO\s*"

def in_cahoots(unit):
    if pd.isna(unit):
        return np.nan
    unit = str(unit)
    if re.fullmatch(cahoots_pattern, unit):
        return "CAHOOTS"
    else:
        return "EPD"

clean.loc[:, 'agency'] = clean['primeunit'].apply(in_cahoots)

#### Time-based variables in minutes and hours to allow for easier analyses later on.

In [8]:
clean.loc[:, 'secs_disp_to_arrv'] = (clean['secs_to_arrv'] - clean['secs_to_disp'])
clean.loc[:, 'mins_disp_to_arrv'] = clean['secs_disp_to_arrv']/60
clean.loc[:, 'hrs_disp_to_arrv'] = clean['secs_disp_to_arrv']/60/60

clean.loc[:, 'hours_to_disp'] = clean['secs_to_disp']/60/60
clean.loc[:, 'hours_to_arrv'] = clean['secs_to_arrv']/60/60

### Fix outliers & missing values

#### Remove missing `secs_to_arrv`, `secs_to_disp`, `secs_to_close`, and `agency`. Only subset calls where 1+ units arrive.

In [9]:
clean_subset = clean[~clean['secs_to_arrv'].isnull()]
clean_subset = clean_subset[~clean_subset['secs_to_disp'].isnull()]
clean_subset = clean_subset[~clean_subset['secs_to_close'].isnull()]
clean_subset = clean_subset[~clean_subset['agency'].isnull()]
clean_subset = clean_subset[clean_subset['units_arrived'] > 0]

#### Remove duplicate row(s).

In [10]:
idx_drop = clean_subset[clean_subset.duplicated(subset=['inci_id'])].index

clean_subset = clean_subset.drop(idx_drop, axis=0)

#### Correct zipcode and replace empty values with `NA`.

In [11]:
clean_subset.loc[clean_subset['zip'] == '94701', 'zip'] = '97401'
clean_subset.loc[clean_subset['zip'] == '', 'zip'] = pd.NA

#### Replace empty closecodes with `NA`.

In [12]:
clean_subset.loc[clean_subset['closecode'] == '', 'closecode'] = pd.NA

#### Subset to only positive `secs_to_arrv` and `secs_to_disp`.

In [13]:
clean_subset = clean_subset[clean_subset['secs_to_arrv'] >= 0]
clean_subset = clean_subset[clean_subset['secs_to_disp'] >= 0]

#### Subset to positive arrival times, and less than 24 hours.

In [14]:
clean_subset = clean_subset[clean_subset['secs_to_disp'] != clean_subset['secs_to_arrv']]
clean_subset = clean_subset[clean_subset['secs_to_arrv'] != clean_subset['secs_to_close']]

clean_subset = clean_subset[clean_subset['secs_disp_to_arrv'] >= 0]
clean_subset = clean_subset[clean_subset['hrs_disp_to_arrv'] <= 24]

#### Subset to calls taking place within 2014 and 2024.

In [15]:
clean_subset = clean_subset[clean_subset['yr'] <= 2024]

### Quick checkpoint to see entries left in cleaned DataFrame

In [16]:
print(f'Percentage of CAD data used: {len(clean_subset)/len(cad)*100:.2f}%')
print(f'Number of entries in cleaned data: {len(clean_subset)}')
print(f'Number of entries in raw CAD data: {len(cad)}')

Percentage of CAD data used: 35.27%
Number of entries in cleaned data: 525041
Number of entries in raw CAD data: 1488577


In [17]:
complete_clean = clean_subset.reset_index(drop=True).copy()

In [18]:
complete_clean

,yr,service,inci_id,calltime,case_id,callsource,nature,closecode,closed_as,secs_to_disp,...,zip,primeunit,units_dispd,units_arrived,agency,secs_disp_to_arrv,mins_disp_to_arrv,hrs_disp_to_arrv,hours_to_disp,hours_to_arrv
0,2014,LAW,14000006,2014-01-01 00:05:17,NaN,PHONE,DISPUTE,ADVI,ADVISED,219.0,...,97401,5E93,3,2,EPD,377.0,6.283333,0.104722,0.060833,0.165556
1,2014,LAW,14000011,2014-01-01 00:10:21,NaN,E911,THEFT,RSLV,RESOLVED,168.0,...,97402,6E45,2,2,EPD,212.0,3.533333,0.058889,0.046667,0.105556
2,2014,LAW,14000014,2014-01-01 00:12:38,NaN,PHONE,ILLEGAL BURNING,ADVI,ADVISED,1160.0,...,97401,7E98,2,2,EPD,102.0,1.700000,0.028333,0.322222,0.350556
3,2014,LAW,14000051,2014-01-01 00:51:42,1400006,W911,ASSAULT,REPT,REPORT TAKEN,132.0,...,97402,5U52,2,2,EPD,491.0,8.183333,0.136389,0.036667,0.173056
4,2014,LAW,14000067,2014-01-01 01:16:15,NaN,PHONE,DISPUTE,ADVI,ADVISED,105.0,...,97402,6E58,2,2,EPD,802.0,13.366667,0.222778,0.029167,0.251944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525036,2024,LAW,24349438,2024-12-31 23:23:53,NaN,W911,SUBJECT SCREAMING,PCHK,PATROL CHECK,115.0,...,97402,5E52,3,3,EPD,142.0,2.366667,0.039444,0.031944,0.071389
525037,2024,LAW,24349443,2024-12-31 23:28:56,NaN,W911,UNKNOWN PROBLEM,PCHK,PATROL CHECK,645.0,...,97401,4E42,2,2,EPD,371.0,6.183333,0.103056,0.179167,0.282222
525038,2024,LAW,24349460,2024-12-31 23:45:33,NaN,PHONE,LOCATION WANTED SUBJECT,FUP,FOLLOW UP INVESTIGATION,7351.0,...,97401,4E14,1,1,EPD,2875.0,47.916667,0.798611,2.041944,2.840556
525039,2024,LAW,24349461,2024-12-31 23:46:01,NaN,PHONE,DISPUTE,PCHK,PATROL CHECK,356.0,...,97403,6E31,3,2,EPD,815.0,13.583333,0.226389,0.098889,0.325278


### Load complete, cleaned DataFrame into csv file named `cleaned_full_class_data.csv`

In [19]:
complete_clean.to_csv('../data/cleaned_full_class_data.csv', index=False)